In [1]:
import os
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/amittimer/DataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "amittimer"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "2f89005b2e2d95e60f8bd873d4818a02bad814f6"

In [2]:
%pwd

'c:\\Users\\amitt\\Documents\\DataScienceProject\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluatinconfig:

    
    root_dir: Path
    model_path: Path
    test_data_path: Path
    metrics_path: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directory,save_json

In [6]:
# updating the configuration manager
class ConfigurationManager:

    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 param_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMAS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_file_path)
        self.schema = read_yaml(schema_file_path)


        create_directory([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluatinconfig:

        config = self.config.model_evaluation

        params = self.param.ElasticNet

        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])


        modelevaluationconfig =  ModelEvaluatinconfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_data_path = config.test_data_path,
            all_params = params,
            metrics_path = config.metrics_path,
            target_column = schema.name,
            mlflow_uri= os.environ['MLFLOW_TRACKING_URI']
        )


        return modelevaluationconfig
    
    

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from mlflow.sklearn import log_model
from urllib.parse import urlparse
import pandas as pd
import joblib
import mlflow
import mlflow.sklearn
import numpy as np

In [8]:
class ModelEvalauation:

    def __init__(self, config: ModelEvaluatinconfig):

        self.config = config

    def eval_metrics(self, actual, pred):

        mse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)


        return mse,mae,r2


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)

        model = joblib.load(self.config.model_path)


        test_x = test_data.drop(self.config.target_column,axis=1)
        test_y = test_data[self.config.target_column]


        mlflow.set_registry_uri(self.config.mlflow_uri)


        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():


            predictions = model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrics(test_y, predictions)

            scores = {"rmse":rmse, "mae":mae, "r2":r2}

            save_json(path=Path(self.config.metrics_path), data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


        if tracking_uri_type_store!="file":

            mlflow.sklearn.log_model(model,"model",registered_model_name="Elasticmodelnew")
        
        else:

            mlflow.sklearn.log_model(model,"model")


In [9]:
try:

    config = ConfigurationManager()

    ModelEvaluatinconfig = config.get_model_evaluation_config()

    ModelEvalauation = ModelEvalauation(ModelEvaluatinconfig)

    ModelEvalauation.log_into_mlflow()

except Exception as e:

    raise e

[2024-11-04 12:13:52,816 - INFO - common - yaml file : config\config.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-04 12:13:52,827 - INFO - common - yaml file : params.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-04 12:13:52,830 - INFO - common - yaml file : schema.yaml loaded successfully]
the content is : {content}, for yaml file : {yaml_file}
[2024-11-04 12:13:52,830 - INFO - common - directory : artifacts created successfully]
[2024-11-04 12:13:52,830 - INFO - common - directory : artifacts/model_evaluation created successfully]
[2024-11-04 12:13:53,627 - INFO - common - json file : artifacts\model_evaluation\metrics.json saved successfully]


2024/11/04 12:13:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run spiffy-hound-554 at: https://dagshub.com/amittimer/DataScienceProject.mlflow/#/experiments/0/runs/cead04f61c9348cdb65e2fb81c915a46.
2024/11/04 12:13:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/amittimer/DataScienceProject.mlflow/#/experiments/0.
2024/11/04 12:13:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Elasticmodelnew'.
2024/11/04 12:14:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Elasticmodelnew, version 1
Created version '1' of model 'Elasticmodelnew'.
